In [2]:
##**Read in csv with diff potential and branch probs for all samples and make a heatmap for cell 
#branch probabilities relative to the normal samples
import numpy as np
import pandas as pd
import scanpy as sc
#from scanpy import utils
import re
import collections
from matplotlib.colors import LinearSegmentedColormap

import sys
import palantir
import os
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use
import pickle
from MulticoreTSNE import MulticoreTSNE as TSNE
from matplotlib import font_manager
import csv

In [3]:
#Set file directory to import from and date
filedir = '/Users/4472241/scCode/palantirEmbeddings/mapToRep1/'
sampleID = 'cmml1'
date = '11-13-2020'

In [4]:
#define function to read in branchProbs and format to percent greater than X (.5)
def csv_import_fn(i, colNames):
    if (i<40):
        sampleID = 'cmml'+str(i)
    else:
        sampleID = 'normal'+str(i-39)
    
    #Import dataframe from r specifying diff potential and branch Prob
    branchProb_diffPot_df = pd.read_csv(filedir+sampleID+'branchProbs_diffPot.csv', index_col=0)

    branchProb_diffPot_df = branchProb_diffPot_df[cols]
    
    #Count how many cells we have
    denominator = branchProb_diffPot_df.count()[1]

    #Count the cells with >50% branch probability for each branch
    gt75 = branchProb_diffPot_df.gt(.5)
    outputArray = gt75.sum()

    #Convert to numpy array and add the unassigned cells to 5th column (HSC Precursors)
    percent = outputArray/denominator
    percentages = np.zeros(8)
    percentages = percent.values

    percentages = np.append(percentages, 1-sum(percentages[1:6]))
    return percentages

In [6]:
#Set oredring to group cmml, hma, rux, normal (normals are last 40+)
ordering = [5,6,7,17,32,33,35,36,37,39,34,38,1,2,3,4,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31, 40, 41, 42, 43,44,45,46]
percentagesNumPy_HSC = np.zeros((len(ordering),8))
cols = ["DiffPot","Ery", "Mono", "CLP", "Mega", "cDC", "pDC"]

count=0
for i in ordering:
    percentagesNumPy_HSC[count,:] = csv_import_fn(i, cols)
    count = count+1


In [7]:
#Subtract average of normals to plot the relative percentages
normalsOnly = percentagesNumPy_HSC[39:,:]
averages = np.mean(normalsOnly,0)

plotRelative = percentagesNumPy_HSC#-averages

In [8]:
#Set figure size and resolution
plt.rcParams['figure.figsize'] = [5, 25]
plt.rcParams['figure.dpi'] = 100

fig, ax = plt.subplots(1,1)
num_rows, num_cols = plotRelative.shape

#Exclude first row (diff potential) to show only branch probablities (not diff potential)
img = ax.imshow(plotRelative[:,1:],extent=[0,num_cols-1,0,num_rows], cmap = 'seismic', clim=[-.5,.5])

#add axis labels and title
plt.xlabel("Cell State (>50% Probability)")
plt.ylabel("Sample")
plt.title("Heatmap of Cell States")

#label samples appropiately to match sample number and cell state
col_labels = np.append(cols, "HSC")
x_label_list = col_labels[1:]
y_label_str = map(str, ordering)
y_label_list = y_label_str

#Set label ticks
ax.set_xticks(np.linspace(0,(num_cols-2),num_cols-1)+.5)
ax.set_yticks(np.linspace((num_rows-1),0,num_rows)+.5)

ax.set_xticklabels(x_label_list)
ax.set_yticklabels(y_label_list)

fig.colorbar(img)

#Save figure
plt.savefig('/Users/4472241/scCode/heatMapRelative.pdf')

findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Bitstream Vera Sans'] not found. Falling back to DejaVu Sans.
